# imports

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import Uncertainty as unc
import UncertaintyM as uncM
import Data.data_provider as dp
import matplotlib.pyplot as plt

# Loading data

In [2]:
data_name = "spambase"
features, target = dp.load_data(data_name)

# split and shuffel the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, shuffle=True, random_state=1)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, shuffle=True, random_state=1)

In [3]:
model = RandomForestClassifier(max_depth=10, n_estimators=10, random_state=1)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
print("model test score = ", model.score(X_test, y_test))

model test score =  0.9217391304347826


In [4]:
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay

model_calib = CalibratedClassifierCV(model, cv=2, method="isotonic")
model_calib.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=RandomForestClassifier(max_depth=10,
                                                             n_estimators=10,
                                                             random_state=1),
                       cv=2, method='isotonic')

In [5]:
model.predict_proba(X_test[:3])

array([[0.0030303 , 0.9969697 ],
       [0.00380489, 0.99619511],
       [0.69982398, 0.30017602]])

In [6]:
model_calib.predict_proba(X_test[:3])

array([[0.02108434, 0.97891566],
       [0.00414938, 0.99585062],
       [0.89047403, 0.10952597]])